# ML Pipeline Preparation

# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with read_sql_table
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sqlalchemy import create_engine
import re 
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle


In [2]:
# load data from database
engine = create_engine('sqlite:///combined.db')
df = pd.read_sql_table('combined', engine)
X = df['message'] # Define feature 
Y = pd.concat([df.iloc[:,4:40],df.iloc[:,-1:]],axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    Doc String: 
    Inputs:
    
    Text Data gets tokenize text , normalize, lemmatize, 
    using the nltk package
    
    Returns: Text data that has been normalize, lemmatize, and tokenize 
        
    """
    # Normalize
    # Set text to lower case and remove punctuation
    text= text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) # Remove punctuation characters
    # Tokenize words 
    tokens = word_tokenize(text)
    # lemmatizer and remove stopwords
    # lemmatizer
    lemmatizer = WordNetLemmatizer()
    # stopwords
    stop_words = set(stopwords.words('english'))
    # lemmatizer and remove stopwords
    output = [lemmatizer.lemmatize(w) for w in tokens if not w in stop_words]
    output = [] 
    for w in tokens: 
        if w not in stop_words: 
            output.append(w)
    return output


### 3. Build a machine learning pipeline
This machine pipeline should take in the message column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# Create pipeline with Classifier
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [6]:
# Train pipeline
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's classification_report on each.

In [7]:
Y_pred = pipeline.predict(X_test) # Predict using pipeline

In [10]:
Y_Predicton_df = pd.DataFrame(Y_pred, columns = Y_test.columns)
# iterating through the columns 
for column in Y_test:
    for i in Y_test.columns:
        print('Category: {}\n'.format(column))
        print(classification_report(Y_test[column],Y_Predicton_df[column])) #calling sklearn's classification_report on each

Category: related

              precision    recall  f1-score   support

           0       0.62      0.46      0.53      1848
           1       0.85      0.91      0.88      5961

    accuracy                           0.80      7809
   macro avg       0.73      0.69      0.70      7809
weighted avg       0.79      0.80      0.79      7809

Category: related

              precision    recall  f1-score   support

           0       0.62      0.46      0.53      1848
           1       0.85      0.91      0.88      5961

    accuracy                           0.80      7809
   macro avg       0.73      0.69      0.70      7809
weighted avg       0.79      0.80      0.79      7809

Category: related

              precision    recall  f1-score   support

           0       0.62      0.46      0.53      1848
           1       0.85      0.91      0.88      5961

    accuracy                           0.80      7809
   macro avg       0.73      0.69      0.70      7809
weighted avg     

### 6. Improve your model
Use grid search to find better parameters.

In [9]:
pipeline.get_params() # get model parameters

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a1f25e598>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_features='auto',
           

#### Looking at the default hyper parameters:

- the n_estimators is set to 'warn' default
- min_samples_split is set to 2.

#### Changing default hyper parameters to improve performanace. 

In [11]:
parameters = {'clf__estimator__n_estimators':[10,20],
              'clf__estimator__max_depth':[5,15],
             'clf__estimator__min_samples_split':[5,10]}

In [12]:
cv = GridSearchCV(pipeline, parameters)

### 7. Test your model

Show the accuracy, precision, and recall of the tuned model.

Since this project focuses on code quality, process, and pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
cv.fit(X_train, Y_train) #training mode using GridSearch

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

### Testing Model

In [14]:
y_pred_2 = cv.predict(X_test) # Preducitng using CV

### models for accuracy, precision

In [15]:
Y_Predicton_df_2 = pd.DataFrame(y_pred_2, columns = Y_test.columns)
# iterating through the columns 
for column in Y_test:
    for i in Y_test.columns:
        print('Catergory: {}\n'.format(column))
        print(classification_report(Y_test[column],Y_Predicton_df_2[column])) #calling sklearn's classification_report on each

Catergory: related

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1848
           1       0.76      1.00      0.87      5961

    accuracy                           0.76      7809
   macro avg       0.38      0.50      0.43      7809
weighted avg       0.58      0.76      0.66      7809

Catergory: related

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1848
           1       0.76      1.00      0.87      5961

    accuracy                           0.76      7809
   macro avg       0.38      0.50      0.43      7809
weighted avg       0.58      0.76      0.66      7809

Catergory: related

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1848
           1       0.76      1.00      0.87      5961

    accuracy                           0.76      7809
   macro avg       0.38      0.50      0.43      7809
weighted avg  

### 8. Try improving your model further. Here are a few ideas:¶
- try other machine learning algorithms
- add other features besides the TF-IDF

In [17]:
# Create a updated RandomForestClassifier with random_state pipeline with Classifier
pipeline_2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state = 5,verbose = 1)))
])

In [18]:
pipeline_2.fit(X_train, Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

In [19]:
Y_pred_3 = pipeline_2.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

In [20]:
Y_Predicton_df_3 = pd.DataFrame(Y_pred_3, columns = Y_test.columns)
# iterating through the columns 
for column in Y_test:
    for i in Y_test.columns:
        print(classification_report(Y_test[column],Y_Predicton_df_3[column])) #calling sklearn's classification_report on each

              precision    recall  f1-score   support

           0       0.62      0.45      0.52      1848
           1       0.84      0.91      0.88      5961

    accuracy                           0.80      7809
   macro avg       0.73      0.68      0.70      7809
weighted avg       0.79      0.80      0.79      7809

              precision    recall  f1-score   support

           0       0.62      0.45      0.52      1848
           1       0.84      0.91      0.88      5961

    accuracy                           0.80      7809
   macro avg       0.73      0.68      0.70      7809
weighted avg       0.79      0.80      0.79      7809

              precision    recall  f1-score   support

           0       0.62      0.45      0.52      1848
           1       0.84      0.91      0.88      5961

    accuracy                           0.80      7809
   macro avg       0.73      0.68      0.70      7809
weighted avg       0.79      0.80      0.79      7809

              preci

### 9. Export your model as a pickle file

In [21]:
# Save model
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete train.py¶
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

#### Reference
- https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
- https://stackoverflow.com/questions/47481874/how-to-get-the-column-name-when-iterating-through-dataframe-pandas
- https://towardsdatascience.com/optimizing-hyperparameters-in-random-forest-classification-ec7741f9d3f6